In [2]:
import numpy as np
import pandas as pd
import os
import re
from glob import glob
from pathlib import Path

In [ ]:
##################
# CT AND BPRNA 
###################

In [21]:
#ct files 
ct_sets = [
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/bid_seq_pos", "label":1, "dataset": "bid_seq_pos"},
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/bid_seq_neg", "label":0, "dataset": "bid_seq_neg"},
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/pseudo_seq_pos", "label":1, "dataset": "pseudo_seq_pos"},
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/pseudo_seq_neg", "label":0, "dataset": "pseudo_seq_neg"},
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/hek_csv_pos", "label":1, "dataset": "hek_csv_pos"},
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/hek_csv_neg", "label":0, "dataset": "hek_csv_neg"},
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/hk_qp", "label":1, "dataset": "hek_qb_pos"},
    {"path": "/nobackup/tjxz93/pse_project/updated_ERNIE/ERNIE-RNA/results/ernie_rna_ss_prediction/bpRNA-1m_test_results/hk_qp_neg", "label":0, "dataset": "hek_qb_neg"},
]


In [22]:
def parse_ct_file(filepath):
    paired = 0
    total = 0
    central_paired = False

    with open(filepath, 'r') as f:
        lines = f.readlines()[1:]  # skip header
        total = len(lines)
        for i, line in enumerate(lines):
            cols = line.strip().split()
            base_idx = int(cols[0])
            pair_idx = int(cols[4])
            if pair_idx != 0:
                paired += 1
                if base_idx == (total // 2 + 1):
                    central_paired = True

    return {
        "sample_id": os.path.basename(filepath).replace(".ct", ""),
        "length": total,
        "n_paired": paired,
        "paired_frac": paired / total,
        "central_paired": int(central_paired)
    }


In [23]:
# Collect CT features and mapping 
all_feats = []
id_to_dataset = {}

for s in ct_sets:
    ct_files = glob(os.path.join(s["path"], "*.ct"))  # match all .ct files
    for f in ct_files:
        try:
            feats = parse_ct_file(f)
            feats["label"] = s["label"]
            feats["dataset"] = s["dataset"]
            all_feats.append(feats)

            sample_id = os.path.basename(f).replace(".ct", "")
            id_to_dataset[sample_id] = s["dataset"]
        except Exception as e:
            print(f"failed to parse {f}: {e}")

ct_df = pd.DataFrame(all_feats)
ct_df.to_csv("/nobackup/tjxz93/pse_project/parsed2_ct_features.csv", index=False)
print("Saved: parsed2_ct_features.csv")


Saved: parsed2_ct_features.csv


In [24]:
import pandas as pd
from pathlib import Path

manifest = pd.read_csv("/nobackup/tjxz93/pse_project/bprna_manifest.csv")
shapes = pd.read_csv("/nobackup/tjxz93/pse_project/bprna_shape_counts.csv")

# normalise paths - consistent paths to match
manifest["st_path"] = manifest["st_path"].apply(lambda p: str(Path(p).resolve()))
shapes["st_path"] = shapes["st_path"].apply(lambda p: str(Path(p).resolve()))

# replace empty in the 'length' column with 121
shapes["length"] = shapes["length"].fillna(121)

# extract label
manifest["label"] = manifest["st_path"].apply(lambda p: Path(p).parts[-2])

# merge
merged = shapes.merge(manifest[["st_path", "label"]], on="st_path", how="left")

print(merged["label"].value_counts())


label
hk_qp_neg         570
hek_csv_neg       412
hek_csv_pos       206
hk_qp_pos         104
bid_seq_neg        34
pseudo_seq_neg     23
bid_seq_pos        14
pseudo_seq_pos      5
Name: count, dtype: int64


In [28]:
# make combiend file
merged.to_csv("bprna_shape_counts_with_labels3.csv", index=False)

In [25]:
print(shapes.columns)


Index(['sample_id', 'length', 'n_stems', 'n_hairpins', 'n_internal',
       'n_multiloops', 'n_bulges', 'n_exterior', 'n_segments', 'n_pseudoknots',
       'pk_total_bp', 'st_path'],
      dtype='object')


In [62]:
bpRNA = pd.read_csv("bprna_shape_counts_with_labels3.csv")
ct = pd.read_csv("parsed2_ct_features.csv")

# check alignment
bpRNA["sample_id"] = bpRNA["sample_id"].astype(str)
ct["sample_id"] = ct["sample_id"].astype(str)

In [63]:

merged_features_str = bpRNA.merge(ct, on="sample_id", how="inner")


In [64]:
print(merged_features_str.columns)

Index(['sample_id', 'length_x', 'n_stems', 'n_hairpins', 'n_internal',
       'n_multiloops', 'n_bulges', 'n_exterior', 'n_segments', 'n_pseudoknots',
       'pk_total_bp', 'st_path', 'label_x', 'length_y', 'n_paired',
       'paired_frac', 'central_paired', 'label_y', 'dataset'],
      dtype='object')


In [65]:
merged_features_str.head(20)

,sample_id,length_x,n_stems,n_hairpins,n_internal,n_multiloops,n_bulges,n_exterior,n_segments,n_pseudoknots,pk_total_bp,st_path,label_x,length_y,n_paired,paired_frac,central_paired,label_y,dataset
0,ENST00000420826:73178913-73179033(-)_zeroshot_...,121.0,6,2,6,3,0,2,3,4,4,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,20,0.165289,0,0,hek_csv_neg
1,ENST00000373232:71993091-71993211(-)_zeroshot_...,121.0,3,2,2,0,0,2,2,2,2,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,10,0.082645,0,0,hek_csv_neg
2,ENST00000534695:43343554-43343674(+)_zeroshot_...,121.0,3,2,2,0,0,2,2,2,2,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,12,0.099174,0,0,hek_csv_neg
3,ENST00000286713:124102758-124102878(-)_zerosho...,121.0,5,2,4,3,0,2,3,1,1,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,12,0.099174,1,0,hek_csv_neg
4,ENST00000367837:135286137-135286257(-)_zerosho...,121.0,5,2,4,3,0,2,3,1,1,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,14,0.115702,0,0,hek_csv_neg
5,ENST00000619304:37570561-37570681(+)_zeroshot_...,121.0,3,1,4,0,0,2,1,1,1,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,12,0.099174,0,0,hek_csv_neg
6,ENST00000409472:1924266-1924386(+)_zeroshot_pr...,121.0,2,2,0,0,0,1,2,0,0,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,4,0.033058,0,0,hek_csv_neg
7,ENST00000551598:49961840-49961960(-)_zeroshot_...,121.0,1,1,0,0,0,2,1,0,0,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,2,0.016529,0,0,hek_csv_neg
8,ENST00000715263:126862253-126862373(+)_zerosho...,121.0,5,1,6,0,1,2,1,1,1,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,16,0.132231,0,0,hek_csv_neg
9,ENST00000588757:40714930-40715050(+)_zeroshot_...,121.0,6,3,4,3,0,2,4,0,0,/Users/beyzaerkal/Desktop/datasets_proj/bpRNA_...,hek_csv_neg,121,14,0.115702,0,0,hek_csv_neg


In [66]:
merged_features_str = merged_features_str.drop(columns=["label_y"])
merged_features_str = merged_features_str.rename(columns={"label_x": "label"})


In [67]:
# check they are teh same
(merged_features_str["length_x"] == merged_features_str["length_y"]).all()


True

In [68]:
merged_features_str = merged_features_str.drop(columns=["length_y"])
merged_features_str = merged_features_str.rename(columns={"length_x": "length"})
merged_features_str = merged_features_str.drop(columns=["dataset", "st_path"])


In [69]:
merged_features_str.head()

,sample_id,length,n_stems,n_hairpins,n_internal,n_multiloops,n_bulges,n_exterior,n_segments,n_pseudoknots,pk_total_bp,label,n_paired,paired_frac,central_paired
0,ENST00000420826:73178913-73179033(-)_zeroshot_...,121.0,6,2,6,3,0,2,3,4,4,hek_csv_neg,20,0.165289,0
1,ENST00000373232:71993091-71993211(-)_zeroshot_...,121.0,3,2,2,0,0,2,2,2,2,hek_csv_neg,10,0.082645,0
2,ENST00000534695:43343554-43343674(+)_zeroshot_...,121.0,3,2,2,0,0,2,2,2,2,hek_csv_neg,12,0.099174,0
3,ENST00000286713:124102758-124102878(-)_zerosho...,121.0,5,2,4,3,0,2,3,1,1,hek_csv_neg,12,0.099174,1
4,ENST00000367837:135286137-135286257(-)_zerosho...,121.0,5,2,4,3,0,2,3,1,1,hek_csv_neg,14,0.115702,0


In [53]:
def clean_sample_id(s):
    if isinstance(s, str):
        if s.startswith("ENST"):
            return re.match(r"^([^_]+)", s).group(1)  # keep up to first underscore
        elif s.startswith("NM_"):
            return re.match(r"^(NM_[0-9.]+)", s).group(1)  # extract NM_ only
    return np.nan  # for missing or malformed entries
    

In [70]:
# keep NM IDs with positions
def clean_sample_id(s):
    if isinstance(s, str):
        if s.startswith("ENST"):
            return re.match(r"^([^_]+)", s).group(1)
        elif s.startswith("NM_"):
            return re.match(r"^(NM_[0-9.]+:[0-9.\-]+\([+-]\))", s).group(1)  # keep full site
    return np.nan


In [71]:
merged_features_str["sample_id_clean"] = merged_features_str["sample_id"].apply(clean_sample_id)

In [72]:
merged_features_str.tail()

,sample_id,length,n_stems,n_hairpins,n_internal,n_multiloops,n_bulges,n_exterior,n_segments,n_pseudoknots,pk_total_bp,label,n_paired,paired_frac,central_paired,sample_id_clean
1391,ENST00000588657.6:317-438(+)_zeroshot_prediction,121.0,4,3,2,0,0,2,3,0,0,bid_seq_neg,14,0.115702,0,ENST00000588657.6:317-438(+)
1392,ENST00000588657.6:317-438(+)_zeroshot_prediction,121.0,4,3,2,0,0,2,3,0,0,bid_seq_neg,14,0.115702,0,ENST00000588657.6:317-438(+)
1393,ENST00000340473.8:246-367(+)_zeroshot_prediction,121.0,4,4,0,0,0,2,4,0,0,bid_seq_neg,10,0.082645,0,ENST00000340473.8:246-367(+)
1394,ENST00000340473.8:246-367(+)_zeroshot_prediction,121.0,4,4,0,0,0,2,4,0,0,bid_seq_neg,10,0.082645,0,ENST00000340473.8:246-367(+)
1395,ENST00000301633.8:280-401(+)_zeroshot_prediction,121.0,5,3,2,0,1,2,3,1,1,bid_seq_neg,12,0.099174,0,ENST00000301633.8:280-401(+)


In [73]:
# check for duplicates
merged_features_str["sample_id_clean"].value_counts()


sample_id_clean
ENST00000340473.8:363-484(+)      4
ENST00000433898.5:43-164(+)       4
ENST00000491306.6:487-608(+)      4
ENST00000616122.5:3978-4099(-)    4
ENST00000395840.6:144-265(+)      4
                                 ..
NM_007236.5:1410-1530(+)          1
NM_145806.4:38-158(+)             1
NM_001379455.1:2073-2193(+)       1
NM_014997.4:3086-3206(+)          1
ENST00000301633.8:280-401(+)      1
Name: count, Length: 1354, dtype: int64

In [74]:
merged_features_str["label"].value_counts()

label
hk_qp_neg         570
hek_csv_neg       412
hek_csv_pos       206
hk_qp_pos         104
bid_seq_neg        48
bid_seq_pos        28
pseudo_seq_neg     23
pseudo_seq_pos      5
Name: count, dtype: int64

In [75]:
merged_features_str = merged_features_str.drop_duplicates()

In [76]:
merged_features_str["label"].value_counts()

label
hk_qp_neg         570
hek_csv_neg       412
hek_csv_pos       206
hk_qp_pos         104
bid_seq_neg        34
pseudo_seq_neg     23
bid_seq_pos        14
pseudo_seq_pos      5
Name: count, dtype: int64

In [77]:
# chekc for duplicates
merged_features_str["sample_id_clean"].value_counts()


sample_id_clean
ENST00000340473.8:363-484(+)      2
ENST00000433898.5:43-164(+)       2
ENST00000491306.6:487-608(+)      2
ENST00000616122.5:3978-4099(-)    2
ENST00000395840.6:144-265(+)      2
                                 ..
NM_007236.5:1410-1530(+)          1
NM_145806.4:38-158(+)             1
NM_001379455.1:2073-2193(+)       1
NM_014997.4:3086-3206(+)          1
ENST00000301633.8:280-401(+)      1
Name: count, Length: 1354, dtype: int64

In [78]:
# save combined str+ classification as csv
merged_features_str.to_csv("bprna_ernie_features3.csv", index=False)
